In [35]:
from astropy.modeling import models
print(models.__file__)

/user/dencheva/conda-dev/envs/jwstdev/lib/python3.5/site-packages/astropy/modeling/models.py


In [36]:
from astropy.modeling.models import *
from astropy.modeling.fitting import *
import numpy as np

## Create input arrays with different shapes

In [37]:
# Create input arrays with different shapes
x = np.arange(4)
x1 = np.array([x, x]).T
x0 = np.array([x, x])
print('x: ', x.shape)
print(x )
print('\n')
print('x1: ', x1.shape)
print(x1)
print('\n')
print('x0: ', x0.shape)
print(x0)

x:  (4,)
[0 1 2 3]


x1:  (4, 2)
[[0 0]
 [1 1]
 [2 2]
 [3 3]]


x0:  (2, 4)
[[0 1 2 3]
 [0 1 2 3]]


# Initialize a model set without passing `n_models`

There are two different ways to initialize the same model set when supplying parameter values - using `n_models` and `model_set_axis`, which I think is a bit confusing.



In [14]:
pn = models.Polynomial1D(1, c0=[1,1], c1=[2,2], n_models=2)
print('axis:', pn.model_set_axis)
print('n_models: ', len(pn))

axis: 0
n_models:  2


In [13]:
pm = models.Polynomial1D(1, c0=[1,1], c1=[2,2], model_set_axis=0)
print('axis:', pm.model_set_axis)
print('n_models: ', len(pm))

axis: 0
n_models:  2


A third and similar way - using the defaults for `n_models` and `model_set_axis` creates a completely different model - a model with array parameters.

In [25]:
pa = models.Polynomial1D(1, c0=[1,1], c1=[2,2])
print('axis:', pa.model_set_axis)
print('n_models: ', len(pa))

axis: False
n_models:  1


I find it confusing to have two flags that control the initialization of model sets. **Isn't it sufficient to require `model_set_axis` and drop `n_models`?** It seems the `n_models` keyword is necessary only when model_set_axis is not defined. Here are examples where the presence of `model_set_axis` determines that a model set has to be initialized. Otherwise a single model is initialized, although it may be with array parameters. Am I missing a use case?

In [49]:
a = np.array([[1,1,1], [2,2,2]])
a.shape

(2, 3)

In [50]:
p = models.Polynomial1D(1, c0=[1,1], c1=[2,2], model_set_axis=0)
print('axis :', p.model_set_axis)
print('# models :', len(p))

axis : 0
# models : 2


In [51]:
p = models.Polynomial1D(1, c0=[[1,1]], c1=[[2,2]], model_set_axis=1)
print('axis :', p.model_set_axis)
print('# models :', len(p))

axis : 1
# models : 2


In [52]:
p = models.Polynomial1D(1, c0=[1,1], c1=[2,2])
print('axis :', p.model_set_axis)
print('# models :', len(p))

axis : False
# models : 1


## Initializing a model set with default values

For polynomials the defaults are automatically constructed as if model_set_axis=0.
For all other models an error is raised.

**This behavior should be consistent - always raise an error or always broadcast parameters.
There's a [request](https://github.com/astropy/astropy/issues/6096) to always broadcast parameters.**

In [53]:
pn = models.Polynomial1D(1, n_models=2)
print('axis:', pn.model_set_axis)
print('n_models: ', len(pn))
pn

axis: 0
n_models:  2


<Polynomial1D(1, c0=[ 0., 0.], c1=[ 0., 0.], n_models=2)>

In [54]:
gn = models.Gaussian1D(n_models=2)
print('axis:', gn.model_set_axis)
print('n_models: ', len(gn))

InputParameterError: All parameter values must be arrays of dimension at least 1 for model_set_axis=0 (the value given for 'mean' is only 0-dimensional)

## Model evaluation

Assume the inputs broadcast with the parameters along the model_set_axis. If not, use a keyword, e.g. **input_axis**, int - to specify the axis of the input, or False - to specify to use the entire input. Is **None** more intuitive than **False**? This is identical to the current behaviour except that the keyword is renamed. 

In [18]:
g1(x1)

array([[ 0.60653066,  0.27067057],
       [ 1.        ,  1.21306132],
       [ 0.60653066,  2.        ],
       [ 0.13533528,  1.21306132]])

# Initializing Model Sets

## 1) Initialize a model set without setting `model_set_axis`
#### Equivalent to model_set_axis=0

In [33]:
# It's set internally to its default value of 0.
p = Polynomial1D(1, n_models=2)
print(p.model_set_axis)
p

0


<Polynomial1D(1, c0=[ 0., 0.], c1=[ 0., 0.], n_models=2)>

In [37]:
# To evaluate this the 0-th axis is used to pass inputs
print(p(x0))

coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [38]:
# If the inputs don't broadcast with the parameters an error is raised
#print(px1))

## 2) Initialize a model set with `model_set_axis=0`
#### Equivalent to 1)

## 3) Initialize a model set with `model_set_axis=1`

In [39]:
# 3) Initialize a model set without setting `model_set_axis=1`.
p1 = Polynomial1D(1, n_models=2, model_set_axis=1)
print(p1.model_set_axis)
p1

1


<Polynomial1D(1, c0=[[ 0., 0.]], c1=[[ 0., 0.]], n_models=2)>

In [40]:
# To evaluate this the 1-th axis is used to pass inputs
print(p1(x1))

coeffs (array([[ 0.,  0.]]), array([[ 0.,  0.]]))
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


## Initialize a model set with model_set_axis=False

In [41]:
# 3) Initialize a model set without setting `model_set_axis=False`.
pf = Polynomial1D(1, n_models=2, model_set_axis=False)
print(pf.model_set_axis)
pf

False


<Polynomial1D(1, c0=[ 0., 0.], c1=[ 0., 0.], n_models=2)>

In [44]:
# To evaluate this the entire input is passed to each model regardless of its shape
print(pf(x1))

coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]]


In [45]:
print(pf(x0))

coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [52]:
x3=np.array([x,x,x])
print('x3.shape: ', x3.shape)
res=pf(x3)
print(res)
print('result.shape: ', res.shape)

x3.shape:  (3, 4)
coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]
result.shape:  (2, 3, 4)


# Change `model_set_axis` when a model is evaluated

In [31]:
p=Polynomial1D(1, c0=[[1,2, 3]], c1=[[3,4, 5]], model_set_axis=1)
x = np.arange(5)

#y = p(x) # error
y = p(x, model_set_axis=False)
y.shape

(5, 3)

In [33]:
p(y) # model_set_axis is 1, so it works

array([[   4.,   10.,   18.],
       [  13.,   26.,   43.],
       [  22.,   42.,   68.],
       [  31.,   58.,   93.],
       [  40.,   74.,  118.]])

In [34]:
p(y.T, model_set_axis=0) # changing to another integer works

array([[   4.,   13.,   22.,   31.,   40.],
       [  10.,   26.,   42.,   58.,   74.],
       [  18.,   43.,   68.,   93.,  118.]])

The example below raises an error.
It can support the case when a model set is created by fitting a model to an array of `y`
but evaluating the fitted model set on the same independent variable. It should be fixed.

In [55]:
# p1(x0, model_set_axis=False)

# p0(x1, model_set_axis=False) already works

Currently it's not possible to fit a model when model_set_axis is set to False. 

This makes sense because a model set requires multile `y` values.

**TODO:** Catch this case and issue an intelligible error.

In [57]:
lfit = LinearLSQFitter()
#lfit(pf, x0, pf(x0))

**TODO:** Explain in the documentation that `model_set_axis=False` is valid only when a model is evaluated.

#### This means we only need to support changing `model_set_axis` from 0 or 1 to False.

## Issues with model sets

In [ ]:
aff=Rotation2D(angle=[1,2], n_models=2)
aff(1,2) # fails

# Should at least set n_models to 1 internally and issue an error when it's not